In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np

In [6]:
gn = pd.read_csv('./path/to/GeneID_list.csv',index_col=0)

In [7]:
df = pd.read_csv("./path/to/WholeData.csv",index_col=0)

In [ ]:
sample_1pg = [i for i in df.columns if i[0]=="1"]
sample_lc = [i for i in df.columns if (i[0]!="1")&(i[:5]!="Hi015")]
sample_sc = [i for i in df.columns if i[:5]=="Hi015"]

In [ ]:
def V_pred(dataset,gene):
    CVm2 = (dataset.loc[gene].std()/dataset.loc[gene].mean())**2
    Em = dataset.loc[gene].mean()
    CVM2 = (dataset.sum(axis=1).std()/dataset.sum(axis=1).mean())**2
    CV1N2 = (dataset.sum(axis=0).std()/dataset.sum(axis=0).mean())**2
    v = CVm2/((1+CVM2)*(1+CV1N2)/Em+CV1N2)
    return v

In [ ]:
vs_1pg = []
for i in tqdm(df[df[sample_1pg].sum(axis=1)>0].index[:]):
    if df.loc[i,sample_1pg].sum()==0:continue
    vs_1pg.append(V_pred(df[df[df.columns].sum(axis=1)>0][sample_1pg],i))

In [ ]:
vs_lc = {}
for i in tqdm(df[df[sample_lc].sum(axis=1)>0].index):
    vs_lc[i] = V_pred(df[df[sample_lc].sum(axis=1)>0][sample_lc],i)

In [ ]:
vs_sc = {}
for i in tqdm(df[df[sample_sc].sum(axis=1)>0].index):
    vs_sc[i] = V_pred(df[df[sample_sc].sum(axis=1)>0][sample_sc],i)

In [ ]:
p_val = 0.05
va_1pg_dropna = [i for i in vs_1pg if not np.isnan(i)]
threshold=sorted(va_1pg_dropna)[::-1][int(len(va_1pg_dropna)*p_val)]
print(threshold)
high_val_genes_sc = [i for i,j in vs_sc.items() if j > threshold]

In [ ]:
p_val = 0.05
va_1pg_dropna = [i for i in vs_1pg if not np.isnan(i)]
threshold=sorted(va_1pg_dropna)[::-1][int(len(va_1pg_dropna)*p_val)]
print(threshold)
high_val_genes_lc = [i for i,j in vs_lc.items() if j > threshold]
print("q_val(FDR)",(p_val*len(va_1pg_dropna))/len(high_val_genes_lc))